# LIBRARIES

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch import optim

import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DATASET

In [3]:

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


# LINEAR NEURAL NET

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
model.to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.25, inplace=False)
    (6): Linear(in_features=512, out_features=10, bias=True)
    (7): ReLU()
  )
)

#  CONVOLUTION NEURAL NETWORK

class ConvNet(nn.Module):
    def __init__(self, input_shape=(1,28,28)):
        super(ConvNet, self).__init__()

        # DEFINE THE CONVOLUTION LAYERS

        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64,3)
        self.conv3 = nn.Conv2d(64, 128, 3)

        # DEFINE THE POOLING LATER

        self.pool = nn.MaxPool2d(2, 2)

        n_size = self._get_conv_output(input_shape)

        # DEFINE THE LINEAR CLASSS

        self.fc1 = nn.Linear(n_size, 512)
        self.fc2 = nn.Linear(512, 10)

        # DEFINE THE DROPOUT
        self.dropout = nn.Dropout(p=0.25)

    def _get_conv_output(self, shape):
        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))
        output_feat = self._forward_features(input)
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size

    def _forward_features(self, X):
        X = self.pool(F.relu(self.conv1(X)))
        X = self.pool(F.relu(self.conv2(X)))
        X = self.pool(F.relu(self.conv3(X)))
        return X

    def forward(self, X):
        X = self._forward_features(X)
        X = X.view(X.size(0), -1)
        X = self.dropout(X) # DROPOUT
        X = F.relu(self.fc1(X))
        X = self.dropout(X) # DROPOUT
        X = self.fc2(X)
        return X
        
model = ConvNet()

# PUSH MODEL TO DEVICE(CUDA)
model.to(device)
print(model)

# HYPERPARAMETERS

In [10]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# TRAINING

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>3f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Metrics: \n Accuracy: {(100*correct):>0.1f}%, Test loss: {test_loss:>8f} \n")

In [12]:
# define loss and optimizer

#loss_fn = nn.NLLLoss()
loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)


In [13]:
%%time
epochs = 10
for t in range(epochs):
    #%%time
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.303405  [    0/60000]
loss: 1.282405  [ 6400/60000]
loss: 0.883138  [12800/60000]
loss: 1.116032  [19200/60000]
loss: 0.880184  [25600/60000]
loss: 0.797371  [32000/60000]
loss: 0.871481  [38400/60000]
loss: 0.764914  [44800/60000]
loss: 0.757430  [51200/60000]
loss: 0.765895  [57600/60000]
Test Metrics: 
 Accuracy: 70.6%, Test loss: 0.799248 

Epoch 2
-------------------------------
loss: 0.580797  [    0/60000]
loss: 0.817575  [ 6400/60000]
loss: 0.596799  [12800/60000]
loss: 0.938537  [19200/60000]
loss: 0.752994  [25600/60000]
loss: 0.685954  [32000/60000]
loss: 0.720032  [38400/60000]
loss: 0.729459  [44800/60000]
loss: 0.778333  [51200/60000]
loss: 0.739541  [57600/60000]
Test Metrics: 
 Accuracy: 72.4%, Test loss: 0.747484 

Epoch 3
-------------------------------
loss: 0.493993  [    0/60000]
loss: 0.735948  [ 6400/60000]
loss: 0.597116  [12800/60000]
loss: 0.927544  [19200/60000]
loss: 0.772045  [25600/60000]
loss: 0.667296  [320